In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cufflinks as cf
cf.go_offline()

In [1]:
from pmdarima import arima
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
from pathlib import Path

In [2]:
from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

In [3]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.svm import SVR
from xgboost import XGBRegressor

In [4]:
from pathlib import Path

In [5]:
# %run ../Library/SVRmape_library.ipynb
%run ../Utils/Utilss.ipynb

## Load data

In [6]:
pathtofile = Path().resolve().parent / "Data" / "Data1.csv"
df = pd.read_csv(pathtofile)[["fecha", "MWh"]].dropna()

In [7]:
df["fecha"] = pd.to_datetime(df["fecha"], format = "%m/%d/%Y")
df.set_index("fecha", inplace = True)
df = df.groupby(pd.Grouper(freq='MS')).agg({"MWh":"mean"})

In [54]:
train = df.query("index < 2018")
test = df.query("index > '2017-12-31'")

In [55]:
df_trf = Utilss.transform_diflog(train, offset = 1)

In [56]:
lag_order = 6
horizon = df.size - df_trf.size

In [57]:
# embed matrix
mbd = Utilss.embed(df_trf, lag_order)

In [58]:
# Splitting to training and testing data
X_train, X_test, y_train, y_test1 = Utilss.train_test(mbd, df, horizon)

In [59]:
y_test = y_test1.to_numpy().reshape(-1)

## Model

### Ridge

In [60]:
hyperparameters = {
    "alpha" : 0.00001
}

predict = Utilss.forecasting(Ridge, X_train, y_train, **hyperparameters)

y_pred = Utilss.backtransform(predict, train, 1)

In [ ]:
df1 = df.copy()
df1["Forecast"] = pd.Series(y_pred.reshape(-1), index = test.index)

In [ ]:
mape = np.mean(np.abs((y_test - y_pred)/y_test))*100
df1.iplot(title = f"MAPE: {mape:.3}%")

### Lasso

In [45]:
hyperparameters = {
    "alpha" : 0.1
}

predict = Utilss.forecasting(Lasso, X_train, y_train, **hyperparameters)

y_pred = Utilss.backtransform(predict, train, 1)

In [46]:
df1 = df.copy()
df1["Forecast"] = pd.Series(y_pred.reshape(-1), index = test.index)

In [47]:
mape = np.mean(np.abs((y_test - y_pred)/y_test))*100
df1.iplot(title = f"MAPE: {mape:.3}%")

### SVR skl

In [48]:
hyperparameters = {
        'kernel' : "rbf",
        'C' : 7.599954, 
        'epsilon' : 0.091463,
        'gamma' : 3.858758,
}

predict = Utilss.forecasting(SVR, X_train, y_train, **hyperparameters)

y_pred = Utilss.backtransform(predict, train, 1)

In [49]:
df1 = df.copy("")
df1["Forecast"] = pd.Series(y_pred.reshape(-1), index = test.index)

In [50]:
mape = np.mean(np.abs((y_test - y_pred)/y_test))*100
df1.iplot(title = f"MAPE: {mape:.3}%")

### XGBoost

In [51]:
hyperparameters = {
    "n_estimators" : 123,
    "max_depth" : 3,
    "objective": "reg:squarederror",
}

predict = Utilss.forecasting(XGBRegressor, X_train, y_train, **hyperparameters)

y_pred = Utilss.backtransform(predict, train, 1)

In [52]:
df1 = df.copy()
df1["Forecast"] = pd.Series(y_pred.reshape(-1), index = test.index)

In [53]:
mape = np.mean(np.abs((y_test - y_pred)/y_test))*100
df1.iplot(title = f"MAPE: {mape:.3}%")